## super()

`super()` is a special function built into the Python language that allows you to use a parent method in the child class.

Its syntax looks like this:

`super(CurrentClassName, instance)`

where the first argument is the name of the class we are currently defining, and the second is an instance of the class we are currently working on (this will usually be `self` when calling `super` in a class definition).

Let's use `super()` with the class we defined in the last lecture: 

In [ ]:
class Leaf(object):
    def __init__(self, color="green"):
        self.color = color
    def fall(self):
        print "Splat!"

class MapleLeaf(Leaf):
    def change_color(self):
        if self.color == "green":
            self.color = "red"
    def fall(self):
        self.change_color()
        super(MapleLeaf, self).fall()

Note that that we are calling the `fall()` method directly on the returned value of the `super` function.

Now our `fall()` method will change color, and then do what the parent's `fall()` method specifies:

In [ ]:
mleaf = MapleLeaf()
mleaf.fall()
print mleaf.color

Now let's introduce the fire concept by writing a `BurnableForest` class that behaves like `Forest` but also has a `fires` array and a `fire_fraction` property. We'll include the methods of `Forest` by *inheritance*.  By inheriting from `Forest` instead of `object`, we get all the methods defined in `Forest`, and we can either overwrite them or add to them.

In [ ]:
import numpy as np

class Forest(object):
    """ A Forest can grow but won't burn.
    """

    def __init__(self, size=(150, 150), p_sapling=0.0025):
        self.size = size
        self.trees = np.zeros(self.size, dtype=bool)
        self.p_sapling = p_sapling

    @property
    def num_cells(self):
        return self.size[0] * self.size[1]

    @property
    def tree_fraction(self):
        return self.trees.sum() / float(self.num_cells)

    def grow_trees(self):
        growth_sites = self._rand_bool(self.p_sapling)
        self.trees[growth_sites] = True

    def _rand_bool(self, p):
        return np.random.uniform(size=self.trees.shape) < p

    def advance_one_step(self):
        self.grow_trees()


class BurnableForest(Forest):
    """ BurnableForest supports fires.
    """

    def __init__(self, p_lightning=5.0e-6, *args, **kwargs):
        super(BurnableForest, self).__init__(*args, **kwargs)
        self.p_lightning = p_lightning
        self.fires = np.zeros(self.size, dtype=bool)

    @property
    def fire_fraction(self):
        return self.fires.sum() / float(self.num_cells)

    def start_fires(self):
        lightning_strikes = self._rand_bool(self.p_lightning) & self.trees
        self.fires[lightning_strikes] = True

    def burn_trees(self):
        """The rules for burning trees:
        -New fires start in tree-filled adjacent cells.
        -Trees in currently burning cells burn and disappear.

        The fire-spreading algorithm is calculated on an array padded
        around the edges with zeros to allow vectorized computations.
        """
        working_size = (self.size[0] + 2, self.size[1] + 2)
        fires = np.zeros(working_size, dtype=bool)
        fires[1:-1, 1:-1] = self.fires
        north = fires[:-2, 1:-1]
        south = fires[2:, 1:-1]
        east = fires[1:-1, :-2]
        west = fires[1:-1, 2:]
        new_fires = (north | south | east | west) & self.trees
        self.trees[self.fires] = False
        self.fires = new_fires

    def advance_one_step(self):
        self.grow_trees()
        self.start_fires()
        self.burn_trees()

Two things are going on here.

1. We're adding an entirely new method that defines the `fire_fraction` property in the same way we defined `tree_fraction`.

2. We're redefining `__init__`; however we don't want to lose the things that `Forest.__init__()` does, so we use the `super()` function.  The signature for `super`, as described above, takes the class and instance as arguments, and it returns the parent class, so you can call any method on that class, using the usual signature.  We use `super()` in the `advance_one_step()` method as well.

We're calling the parent constructor first, and then we add another attribute to track the fires.  I used `*args` and `**kwargs` so that I don't have to update the signature in `BurnableForest` if I decide to change it for `Forest`.

We also moved the other methods specific to fire behavior to the `BurnableForest` class.

In [ ]:
burnable_forest = BurnableForest()
burnable_forest.grow_trees()
burnable_forest.start_fires()
print burnable_forest.tree_fraction
print burnable_forest.fire_fraction

In [ ]:
from matplotlib.pyplot import plot
%matplotlib inline

forest = Forest()
burnable_forest = BurnableForest()
tree_history = []
for i in range(2500):
    forest.advance_one_step()
    burnable_forest.advance_one_step()
    tree_history.append((forest.tree_fraction, burnable_forest.tree_fraction))
    
plot(tree_history)

Now we can see that our `Forest` will continue growing towards 100% tree fraction, while our `BurnableForest` will show oscillations due to fires.

Copyright 2008-2016, Enthought, Inc.<br>Use only permitted under license.  Copying, sharing, redistributing or other unauthorized use strictly prohibited.<br>http://www.enthought.com